In [3]:
from utils.clearml_module import clone_template

In [2]:
# Проверка, создание единственного клона

dataset_hyper_params_dict = {'business_unit': ['1010']#, '1020', '1030', '1060', '1110', '1240', '3990', '5060'],
                                           ,"analog_group": ['150']#, '300-40', '400-40', '50-40', '500-40'],
                                           ,"window_size": [60]#, 120, 360],
                                           ,"split_date": ["01.01.2021"]
                                           # "n_predict" : 1,
                                           ,"target_column_name_formatter": ['predict_{}']#, 'predict_in_train_{}', 'predict_normalized_{}'],
                                           ,"log_target": [False]#, True],
                                           ,"scaling": ["Standard"]#, 'MinMax'],
                                        #    "scaler_kwargs": None,
                                           # "drop_not_scalable": True,
                                           }

model_type_hyper_params_dict = {
            "model_type": ["RandomForestRegressor"]#, "MLPRegressor", "Lasso", "ElasticNet"],
            ,"use_kfold": [True]#, False],
            ,"save_kfold_predicts": [True],
            # "save_model": False,
        }

kflod_kwargs_hyper_params_dict = {
            "n_splits": [5],
            "shuffle": [False]
        }


In [10]:
dataset_hyper_params_dict = {'business_unit': ['1010', '1020', '1030', '1060', '1110', '1240', '3990', '5060']
                                        #    ,"analog_group": ['150-40', '200-40', '300-40', '400-40', '50-40', '500-40']
    #                                        ,"analog_group": ['15', '20', '25', '32', '40', '50', '65', '80', '100', '150',
    #    '200', '250', '300', '350', '400', '500', '700', '600', '800',
    #    '1000']
    ,"analog_group": ['50', '100', '150', '200', '250', '300']
                                           ,"window_size": [60, 120]#, 360],
                                           ,"split_date": ["01.01.2021"]
                                           # "n_predict" : 1,
                                           ,"target_column_name_formatter": ['predict_{}']#, 'predict_in_train_{}', 'predict_normalized_{}'],
                                           ,"log_target": [False]#, True],
                                           ,"scaling": ["Standard"]#, 'MinMax'],
                                        #    "scaler_kwargs": None,
                                           # "drop_not_scalable": True,
                                           }

model_type_hyper_params_dict = {
            # "model_type": ["Ridge", "RandomForestRegressor", "CatBoostRegressor", "MLPRegressor", "Lasso", "ElasticNet"]
            "model_type": ["Ridge", "RandomForestRegressor", "CatBoostRegressor"]#, "MLPRegressor", "Lasso", "ElasticNet"]
            # ,"use_kfold": [True, False]
            ,"use_kfold": [False]
            ,"save_kfold_predicts": [True]
            # "save_model": False,
        }

kflod_kwargs_hyper_params_dict = {
            "n_splits": [5],
            "shuffle": [False]
        }

In [3]:
# 0307

dataset_hyper_params_dict = {'business_unit': ['1010', '1020', '1030', '1060', '1110', '1240', '3990', '5060']
                                        #    ,"analog_group": ['150-40', '200-40', '300-40', '400-40', '50-40', '500-40']
    #                                        ,"analog_group": ['15', '20', '25', '32', '40', '50', '65', '80', '100', '150',
    #    '200', '250', '300', '350', '400', '500', '700', '600', '800',
    #    '1000']
    ,"analog_group": ['50', '100', '150', '200', '250', '300']
                                           ,"window_size": [60, 120, 360]
                                           ,"split_date": ["01.01.2021"]
                                           # "n_predict" : 1,
                                           ,"target_column_name_formatter": ['predict_{}']#, 'predict_in_train_{}', 'predict_normalized_{}']
                                           ,"target_column_name_formatter": ['predict_{}', 'predict_normalized_{}']
                                           ,"log_target": [False, True]
                                           ,"scaling": ["Standard", 'MinMax'],
                                        #    "scaler_kwargs": None,
                                           # "drop_not_scalable": True,
                                           }

model_type_hyper_params_dict = {
            # "model_type": ["Ridge", "RandomForestRegressor", "CatBoostRegressor", "MLPRegressor", "Lasso", "ElasticNet"]
            "model_type": ["Ridge", "RandomForestRegressor", "CatBoostRegressor"]#, "MLPRegressor", "Lasso", "ElasticNet"]
            # ,"use_kfold": [True, False]
            # ,"use_kfold": [False]
            ,"use_kfold": [True]
            ,"save_kfold_predicts": [True]
            # "save_model": False,
        }

kflod_kwargs_hyper_params_dict = {
            "n_splits": [5],
            "shuffle": [False]
        }

In [8]:
from clearml import Task
from itertools import product
from sklearn.model_selection import ParameterGrid

In [22]:
def clone_template(template_task_id, dataset_hyper_params_dict, model_type_hyper_params_dict, kflod_kwargs_hyper_params_dict, model_kwargs_params_dict=None,  queue_name='default'):
    template_task = Task.get_task(task_id=template_task_id)
    if model_kwargs_params_dict is None:
        param_grid = list(product(ParameterGrid(dataset_hyper_params_dict),
                            ParameterGrid(model_type_hyper_params_dict),
                            ParameterGrid(kflod_kwargs_hyper_params_dict), 
                            [None]))
    else:
        param_grid = list(product(ParameterGrid(dataset_hyper_params_dict),
                            ParameterGrid(model_type_hyper_params_dict),
                            ParameterGrid(kflod_kwargs_hyper_params_dict),
                            ParameterGrid(model_kwargs_params_dict)))

    total_len = len(list(param_grid))
    print(f"Total amount of clones = {total_len}")
    i = 0

    for dataset_param_grid, model_type_param_grid, kfold_param_grid, model_kwargs_params_grid in param_grid:
        i += 1
        print(f"pair {i}/{total_len}: " , end='')
        pair = (dataset_param_grid['business_unit'],
                dataset_param_grid['analog_group'])
        window_size = dataset_param_grid['window_size']
        model_name = model_type_param_grid['model_type']
        print("{}, window_size: {}, model_name: {}".format(
            *map(repr, (pair, window_size, model_name))))

        cloned_task = Task.clone(source_task=template_task)
        cloned_task.add_tags(
            ["grid_search", model_name, f"bu={pair[0]}", f"group={pair[1]}", f"window_size={window_size}"])

        # cloned_task.set_parameter(f"model_type/model_type", value=model_name)
        for key, value in dataset_param_grid.items():
            cloned_task.set_parameter(f"dataset_params/{key}", value=value)

        for key, value in model_type_param_grid.items():
            cloned_task.set_parameter(f"model_type_params/{key}", value=value)

        for key, value in kfold_param_grid.items():
            cloned_task.set_parameter(
                f"kflod_kwargs_params/{key}", value=value)

        if model_kwargs_params_dict is not None:
            for key, value in model_kwargs_params_grid.items():
                cloned_task.set_parameter(
                    f"model_kwargs_params/{key}", value=value)

        Task.enqueue(task=cloned_task, queue_name=queue_name)

In [25]:
# 0407

dataset_hyper_params_dict = {
    # 'business_unit': ['1010', '1020', '1030', '1060', '1110', '1240', '3990', '5060']
    'business_unit': ['all']
                                        #    ,"analog_group": ['150-40', '200-40', '300-40', '400-40', '50-40', '500-40']
    #                                        ,"analog_group": ['15', '20', '25', '32', '40', '50', '65', '80', '100', '150',
    #    '200', '250', '300', '350', '400', '500', '700', '600', '800',
    #    '1000']
    ,"analog_group": ['50', '100', '150', '200', '250', '300', '400', '500']
                                           ,"window_size": [60, 120]#, 360]
                                           ,"split_date": ["01.01.2021"]
                                           # "n_predict" : 1,
                                        #    ,"target_column_name_formatter": ['predict_{}']#, 'predict_in_train_{}', 'predict_normalized_{}']
                                           ,"target_column_name_formatter": ['predict_{}', 'predict_normalized_{}']
                                           ,"log_target": [False, True]
                                           ,"scaling": ["Standard", 'MinMax'],
                                        #    "scaler_kwargs": None,
                                           # "drop_not_scalable": True,
                                           }

model_type_hyper_params_dict = {
            # "model_type": ["Ridge", "RandomForestRegressor", "CatBoostRegressor", "MLPRegressor", "Lasso", "ElasticNet"]
            "model_type": ["RandomForestRegressor"]#, "MLPRegressor", "Lasso", "ElasticNet"]
            # ,"use_kfold": [True, False]
            # ,"use_kfold": [False]
            # ,"use_kfold": [True]
            ,"use_kfold": [False]
            ,"save_kfold_predicts": [True]
            # "save_model": False,
        }

model_kwargs_params_dict = {
    "n_estimators": [50, 100, 200]
    , "max_depth": [10, 20]
    }

kflod_kwargs_hyper_params_dict = {
            "n_splits": [5],
            "shuffle": [False]
        }

In [23]:
# # 0407 test

# dataset_hyper_params_dict = {
#     # 'business_unit': ['1010', '1020', '1030', '1060', '1110', '1240', '3990', '5060']
#     'business_unit': ['all']
#                                         #    ,"analog_group": ['150-40', '200-40', '300-40', '400-40', '50-40', '500-40']
#     #                                        ,"analog_group": ['15', '20', '25', '32', '40', '50', '65', '80', '100', '150',
#     #    '200', '250', '300', '350', '400', '500', '700', '600', '800',
#     #    '1000']
#     ,"analog_group": ['50']#, '100', '150', '200', '250', '300', '400', '500']
#                                            ,"window_size": [60]#, 120]#, 360]
#                                            ,"split_date": ["01.01.2021"]
#                                            # "n_predict" : 1,
#                                         #    ,"target_column_name_formatter": ['predict_{}']#, 'predict_in_train_{}', 'predict_normalized_{}']
#                                            ,"target_column_name_formatter": ['predict_{}']#, 'predict_normalized_{}']
#                                            ,"log_target": [False]#, True]
#                                            ,"scaling": ["Standard"]#, 'MinMax'],
#                                         #    "scaler_kwargs": None,
#                                            # "drop_not_scalable": True,
#                                            }

# model_type_hyper_params_dict = {
#             # "model_type": ["Ridge", "RandomForestRegressor", "CatBoostRegressor", "MLPRegressor", "Lasso", "ElasticNet"]
#             "model_type": ["RandomForestRegressor"]#, "MLPRegressor", "Lasso", "ElasticNet"]
#             # ,"use_kfold": [True, False]
#             # ,"use_kfold": [False]
#             ,"use_kfold": [False]
#             ,"save_kfold_predicts": [True]
#             # "save_model": False,
#         }

# model_kwargs_params_dict = {
#     "n_estimators": [50]#, 100, 200]
#     , "max_depth": [10]#, 20]
#     }

# kflod_kwargs_hyper_params_dict = {
#             "n_splits": [5],
#             "shuffle": [False]
#         }

In [6]:
# 0507 catboost

dataset_hyper_params_dict = {
    # 'business_unit': ['1010', '1020', '1030', '1060', '1110', '1240', '3990', '5060']
    'business_unit': ['all']
                                        #    ,"analog_group": ['150-40', '200-40', '300-40', '400-40', '50-40', '500-40']
    #                                        ,"analog_group": ['15', '20', '25', '32', '40', '50', '65', '80', '100', '150',
    #    '200', '250', '300', '350', '400', '500', '700', '600', '800',
    #    '1000']
    ,"analog_group": ['50', '100', '150', '200', '250', '300', '400', '500']
                                           ,"window_size": [120]#, 360]
                                           ,"split_date": ["01.01.2021"]
                                           # "n_predict" : 1,
                                        #    ,"target_column_name_formatter": ['predict_{}']#, 'predict_in_train_{}', 'predict_normalized_{}']
                                           ,"target_column_name_formatter": ['predict_normalized_{}']
                                           ,"log_target": [False]
                                           ,"scaling": ["Standard"],
                                        #    "scaler_kwargs": None,
                                           # "drop_not_scalable": True,
                                           }

model_type_hyper_params_dict = {
            # "model_type": ["Ridge", "RandomForestRegressor", "CatBoostRegressor", "MLPRegressor", "Lasso", "ElasticNet"]
            "model_type": ["CatBoostRegressor"]#, "MLPRegressor", "Lasso", "ElasticNet"]
            # ,"use_kfold": [True, False]
            # ,"use_kfold": [False]
            # ,"use_kfold": [True]
            ,"use_kfold": [False]
            ,"save_kfold_predicts": [True]
            # "save_model": False,
        }

# model_kwargs_params_dict = {
#     "n_estimators": [50, 100, 200]
#     , "max_depth": [10, 20]
#     }

model_kwargs_params_dict = {
    "max_depth": [2, 5, 10]
    , "n_estimators": [500, 1000]
    , "l2_leaf_reg": [0.01, 0.1, 1] 
    }

kflod_kwargs_hyper_params_dict = {
            "n_splits": [5],
            "shuffle": [False]
        }

In [1]:
# 0507 catboost test

dataset_hyper_params_dict = {
    # 'business_unit': ['1010', '1020', '1030', '1060', '1110', '1240', '3990', '5060']
    'business_unit': ['all']
                                        #    ,"analog_group": ['150-40', '200-40', '300-40', '400-40', '50-40', '500-40']
    #                                        ,"analog_group": ['15', '20', '25', '32', '40', '50', '65', '80', '100', '150',
    #    '200', '250', '300', '350', '400', '500', '700', '600', '800',
    #    '1000']
    ,"analog_group": ['50']#, '100', '150', '200', '250', '300', '400', '500']
                                           ,"window_size": [120]#, 360]
                                           ,"split_date": ["01.01.2021"]
                                           # "n_predict" : 1,
                                        #    ,"target_column_name_formatter": ['predict_{}']#, 'predict_in_train_{}', 'predict_normalized_{}']
                                           ,"target_column_name_formatter": ['predict_normalized_{}']
                                           ,"log_target": [False]
                                           ,"scaling": ["Standard"],
                                        #    "scaler_kwargs": None,
                                           # "drop_not_scalable": True,
                                           }

model_type_hyper_params_dict = {
            # "model_type": ["Ridge", "RandomForestRegressor", "CatBoostRegressor", "MLPRegressor", "Lasso", "ElasticNet"]
            "model_type": ["CatBoostRegressor"]#, "MLPRegressor", "Lasso", "ElasticNet"]
            # ,"use_kfold": [True, False]
            # ,"use_kfold": [False]
            # ,"use_kfold": [True]
            ,"use_kfold": [False]
            ,"save_kfold_predicts": [True]
            # "save_model": False,
        }

# model_kwargs_params_dict = {
#     "n_estimators": [50, 100, 200]
#     , "max_depth": [10, 20]
#     }

model_kwargs_params_dict = {
    "max_depth": [5]#, 10, 20]
    , "n_estimators": [200]#, 500]
    , "l2_leaf_reg": [0.01]#, 0.1, 1] 
    }

kflod_kwargs_hyper_params_dict = {
            "n_splits": [5],
            "shuffle": [False]
        }

In [7]:
# 0707 catboost

dataset_hyper_params_dict = {
    # 'business_unit': ['1010', '1020', '1030', '1060', '1110', '1240', '3990', '5060']
    'business_unit': ['all']
                                        #    ,"analog_group": ['150-40', '200-40', '300-40', '400-40', '50-40', '500-40']
    #                                        ,"analog_group": ['15', '20', '25', '32', '40', '50', '65', '80', '100', '150',
    #    '200', '250', '300', '350', '400', '500', '700', '600', '800',
    #    '1000']
    ,"analog_group": ['50', '100', '150', '200', '250', '300', '400', '500']
    # ,"analog_group": ['50-40', '100-40', '150-40', '200-40', '250-40', '300-40', '400-40', '500-40']
                                        #    ,"window_size": [60, 120]#, 360]
                                           ,"window_size": [60, 120, 360]
                                           ,"split_date": ["01.01.2021"]
                                           # "n_predict" : 1,
                                        #    ,"target_column_name_formatter": ['predict_{}']#, 'predict_in_train_{}', 'predict_normalized_{}']
                                           ,"target_column_name_formatter": ['predict_normalized_{}']
                                        #    ,"log_target": [False]
                                          #  ,"log_target": [True]
                                           ,"log_target": [True, False]

                                           ,"scaling": ["Standard"],
                                        #    "scaler_kwargs": None,
                                           # "drop_not_scalable": True,
                                           }

model_type_hyper_params_dict = {
            # "model_type": ["Ridge", "RandomForestRegressor", "CatBoostRegressor", "MLPRegressor", "Lasso", "ElasticNet"]
            "model_type": ["CatBoostRegressor"]#, "MLPRegressor", "Lasso", "ElasticNet"]
            # ,"use_kfold": [True, False]
            # ,"use_kfold": [False]
            # ,"use_kfold": [True]
            ,"use_kfold": [False]
            ,"save_kfold_predicts": [True]
            # "save_model": False,
        }

# model_kwargs_params_dict = {
#     "n_estimators": [50, 100, 200]
#     , "max_depth": [10, 20]
#     }

model_kwargs_params_dict = {
    "max_depth": [2, 5, 10]
    , "n_estimators": [500, 1000]
    , "l2_leaf_reg": [0.01, 0.1, 1] 
    }

kflod_kwargs_hyper_params_dict = {
            "n_splits": [5],
            "shuffle": [False]
        }

In [5]:
# 0707 catboost test

dataset_hyper_params_dict = {
    # 'business_unit': ['1010', '1020', '1030', '1060', '1110', '1240', '3990', '5060']
    'business_unit': ['all']
                                        #    ,"analog_group": ['150-40', '200-40', '300-40', '400-40', '50-40', '500-40']
    #                                        ,"analog_group": ['15', '20', '25', '32', '40', '50', '65', '80', '100', '150',
    #    '200', '250', '300', '350', '400', '500', '700', '600', '800',
    #    '1000']
    ,"analog_group": ['50']#, '100', '150', '200', '250', '300', '400', '500']
                                           ,"window_size": [120]#, 360]
                                           ,"split_date": ["01.01.2021"]
                                           # "n_predict" : 1,
                                        #    ,"target_column_name_formatter": ['predict_{}']#, 'predict_in_train_{}', 'predict_normalized_{}']
                                           ,"target_column_name_formatter": ['predict_normalized_{}']
                                        #    ,"log_target": [False]
                                           ,"log_target": [True]
                                        #    ,"log_target": [True, False]
                                           ,"scaling": ["Standard"],
                                        #    "scaler_kwargs": None,
                                           # "drop_not_scalable": True,
                                           }

model_type_hyper_params_dict = {
            # "model_type": ["Ridge", "RandomForestRegressor", "CatBoostRegressor", "MLPRegressor", "Lasso", "ElasticNet"]
            "model_type": ["CatBoostRegressor"]#, "MLPRegressor", "Lasso", "ElasticNet"]
            # ,"use_kfold": [True, False]
            # ,"use_kfold": [False]
            # ,"use_kfold": [True]
            ,"use_kfold": [False]
            ,"save_kfold_predicts": [True]
            # "save_model": False,
        }

# model_kwargs_params_dict = {
#     "n_estimators": [50, 100, 200]
#     , "max_depth": [10, 20]
#     }

model_kwargs_params_dict = {
    "max_depth": [5]#, 10, 20]
    , "n_estimators": [200]#, 500]
    , "l2_leaf_reg": [0.01]#, 0.1, 1] 
    }

kflod_kwargs_hyper_params_dict = {
            "n_splits": [5],
            "shuffle": [False]
        }

In [4]:
# task_id = "3e10e598ce6746dabb4b9ef915aeb8c1" # 0107
# task_id = "0127454e4fa147c29c3cbcfca99d7f3f" # 0407
# task_id = "28a03ce4299e4194becb79f02b63f445" # 0407
# task_id = "34a3e5319922464d8367e7f730e2015c" # 0407
# task_id = "8a0ca92efad04c49acc144d0ada6dc23" # 0407
# task_id = "b5f9c93309b04ebc98497080d4584b0b" # 0407
# task_id = "1993531386e34a5aba27cfe18cc71676" # 0507
# task_id = "8df74f49973d41369905f2596ad67914" # 0507
# task_id = "94cc73c3a1d04790a4c2fce52f100a41" # 0507
# task_id = "9950fc7379eb4cd29d95701c713d3444" # 0507

# task_id = "194f018062524d7b8ac1187ff7800bfe" # 0507 test
# task_id = "55191e00b0f9491bb5b1d1b684dc842f" # 0707 test

# task_id = "bbf1591a31fe4e11ae61de0431734b5a" # 0707 pn40
# task_id = "67d159a7b5cc46aea8c8bf7f7f4bdba5" # 0707 pn40

task_id = "0f0c9e2eb46a4b81bc69030d5aa353ca" # 0907 pn40

clone_template(
    template_task_id=task_id,
    dataset_hyper_params_dict=dataset_hyper_params_dict,
    model_type_hyper_params_dict=model_type_hyper_params_dict,
    kflod_kwargs_hyper_params_dict=kflod_kwargs_hyper_params_dict,
    model_kwargs_params_dict=model_kwargs_params_dict,
    #  queue_name='megion-cpu')
    # queue_name='default')
    queue_name='zra_cpu')


Total amount of clones = 576
pair 1/576: ('all', '50-40'), window_size: 60, model_name: 'CatBoostRegressor'
pair 2/576: ('all', '50-40'), window_size: 60, model_name: 'CatBoostRegressor'
pair 3/576: ('all', '50-40'), window_size: 60, model_name: 'CatBoostRegressor'
pair 4/576: ('all', '50-40'), window_size: 60, model_name: 'CatBoostRegressor'
pair 5/576: ('all', '50-40'), window_size: 60, model_name: 'CatBoostRegressor'
pair 6/576: ('all', '50-40'), window_size: 60, model_name: 'CatBoostRegressor'
pair 7/576: ('all', '50-40'), window_size: 60, model_name: 'CatBoostRegressor'
pair 8/576: ('all', '50-40'), window_size: 60, model_name: 'CatBoostRegressor'
pair 9/576: ('all', '50-40'), window_size: 60, model_name: 'CatBoostRegressor'
pair 10/576: ('all', '50-40'), window_size: 60, model_name: 'CatBoostRegressor'
pair 11/576: ('all', '50-40'), window_size: 60, model_name: 'CatBoostRegressor'
pair 12/576: ('all', '50-40'), window_size: 60, model_name: 'CatBoostRegressor'
pair 13/576: ('all',

In [8]:
# task_id = "3e10e598ce6746dabb4b9ef915aeb8c1" # 0107
# task_id = "0127454e4fa147c29c3cbcfca99d7f3f" # 0407
# task_id = "28a03ce4299e4194becb79f02b63f445" # 0407
# task_id = "34a3e5319922464d8367e7f730e2015c" # 0407
# task_id = "8a0ca92efad04c49acc144d0ada6dc23" # 0407
# task_id = "b5f9c93309b04ebc98497080d4584b0b" # 0407
# task_id = "1993531386e34a5aba27cfe18cc71676" # 0507
# task_id = "8df74f49973d41369905f2596ad67914" # 0507
# task_id = "94cc73c3a1d04790a4c2fce52f100a41" # 0507
# task_id = "9950fc7379eb4cd29d95701c713d3444" # 0507

# task_id = "194f018062524d7b8ac1187ff7800bfe" # 0507 test
# task_id = "55191e00b0f9491bb5b1d1b684dc842f" # 0707 test

# task_id = "bbf1591a31fe4e11ae61de0431734b5a" # 0707 pn40
# task_id = "67d159a7b5cc46aea8c8bf7f7f4bdba5" # 0707 pn40

# task_id = "0f0c9e2eb46a4b81bc69030d5aa353ca" # 0907 pn40
task_id = "460041dff4bd4836a6e0f0e5ee09f6d1" # 0907

clone_template(
    template_task_id=task_id,
    dataset_hyper_params_dict=dataset_hyper_params_dict,
    model_type_hyper_params_dict=model_type_hyper_params_dict,
    kflod_kwargs_hyper_params_dict=kflod_kwargs_hyper_params_dict,
    model_kwargs_params_dict=model_kwargs_params_dict,
    #  queue_name='megion-cpu')
    # queue_name='default')
    queue_name='zra_cpu')


Total amount of clones = 864
pair 1/864: ('all', '50'), window_size: 60, model_name: 'CatBoostRegressor'
pair 2/864: ('all', '50'), window_size: 60, model_name: 'CatBoostRegressor'
pair 3/864: ('all', '50'), window_size: 60, model_name: 'CatBoostRegressor'
pair 4/864: ('all', '50'), window_size: 60, model_name: 'CatBoostRegressor'
pair 5/864: ('all', '50'), window_size: 60, model_name: 'CatBoostRegressor'
pair 6/864: ('all', '50'), window_size: 60, model_name: 'CatBoostRegressor'
pair 7/864: ('all', '50'), window_size: 60, model_name: 'CatBoostRegressor'
pair 8/864: ('all', '50'), window_size: 60, model_name: 'CatBoostRegressor'
pair 9/864: ('all', '50'), window_size: 60, model_name: 'CatBoostRegressor'
pair 10/864: ('all', '50'), window_size: 60, model_name: 'CatBoostRegressor'
pair 11/864: ('all', '50'), window_size: 60, model_name: 'CatBoostRegressor'
pair 12/864: ('all', '50'), window_size: 60, model_name: 'CatBoostRegressor'
pair 13/864: ('all', '50'), window_size: 60, model_name: